<a href="https://colab.research.google.com/github/Rishika70/LLM/blob/main/Prompt_Engineering_Mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Libraries

### Huggingface libraries to use Mistral 7B LLM (Open Source LLM)

### LangChain library to call LLM to generate response based on the prompt

In [37]:

#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:

# Workaround to avoid following error at notebook
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [39]:

# Huggingface libraries to run LLM.
# We will understand in depth later
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

#LangChain related libraries
!pip install -q -U langchain


In [40]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cuda
Tesla T4


# We are using Mistral 7 Billion parameter LLM (Open source from HuggingFace)

In [43]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

# Creating pipelines to run LLM at Colab notebook

## Huggingface transformers pipeline
This is an object that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.

We are using "text-generation" task. It's sort of chatGPT where you ask question and model respond with answer of that question based on it's intelligence.

LangChain HuggingFacePipeline
Integrating "transformers.pipeline" with LangChain.

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
    temperature = 0.5,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

## Fire away with your question prompt, and get ready to be spellbound by the magic that follows!

In [ ]:

text = "What is the purpose of our life in 100 words?"
response = mistral_llm.invoke(text)
print(response)


Answer: The purpose of our life is to seek happiness, fulfill our potential, and make a positive impact on the world. We can achieve this by pursuing our passions, developing meaningful relationships, learning from our experiences, and striving for personal growth. Ultimately, the purpose of our life is to live it to the fullest and leave a lasting legacy that inspires future generations.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

#### Prompt Template
prompt_template = """[INST] You are a reliable and trustworthy assistant, providing helpful and respectful responses that precisely address the context.
Answer the question below from context below :
{context}
{question} [/INST]
"""

question = """When did Virat Kohli first win?"""
context = """ Virat Kohli was born on November 5, 1988, in Delhi, India. He grew up in Delhi and was one of the first to train at the West Delhi Cricket Academy, created in 1998. In 2002 he played for the Delhi Under-15 team and was the highest run scorer in the 2003–04 Vijay Merchant Trophy, playing for the Delhi Under-17 team.

In February 2006 Kohli made his domestic debut for Delhi in a one-day match against Services (a team representing the Indian armed forces) but did not get a chance to bat. He scored only 10 runs in his first-class debut (first-class cricket refers to matches that last three or more days and feature two sides of 11 players each) against Tamil Nadu in November that year. He scored 90 runs in difficult conditions in a first-class match against Karnataka in December, helping Delhi draw the Test. In April 2007 he scored 35 runs in his T20 domestic debut against Himachal Pradesh.

Kohli captained the Under-19 Indian cricket team to victory at the ICC Under-19 World Cup in Kuala Lumpur, Malaysia, in 2008. His exploits were rewarded with an IPL contract from RCB for $30,000. He also made his international debut in an ODI that year, opening the batting and scoring 12 in a defeat of Sri Lanka in Dambulla, Sri Lanka. In 2009 he scored 405 runs in nine innings in the Emerging Players Tournament in Australia, ensuring that he would be at the top of the national team selectors’ minds."""

prompt = PromptTemplate(template=prompt_template, input_variables=["question","context"])

llm_chain = LLMChain(prompt=prompt, llm=mistral_llm)

response = llm_chain.invoke({"question":question,"context":context})

print(response['text'])


Virat Kohli won his first game as captain of the Indian national team in 2012 when they defeated England in an ODI match held in Kolkata, India. This was just a few months after he had taken over the captaincy from MS Dhoni.
